# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
from utils import get_openai_api_key
import os
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": os.getenv('CHAT_MODEL'), "base_url": os.getenv('OPENAI_API_BASE')}

## The task!

In [2]:
task = '''
        撰写一篇简洁而引人入胜的博客文章，主题为
       DeepLearning.AI。确保博客文章在
       100个字以内。
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="你是一位作家。你需要撰写引人入胜且简洁的博客文章（包含标题），主题由给定内容决定。你必须根据收到的反馈来润色你的写作，并提供一个精炼的版本。只返回你的最终作品，不附加任何评论。",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

标题：探索DeepLearning.AI：智能的未来

在DeepLearning.AI的引领下，人工智能的边界正不断扩展。这个平台通过提供前沿的课程和资源，帮助学习者掌握深度学习的精髓。无论你是AI新手还是专家，DeepLearning.AI都为你提供了通往智能未来的桥梁。加入我们，一起揭开AI的神秘面纱，探索技术的无限可能。


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="你是一位评论家。你需要审查写作的质量，并提供有建设性的反馈，以帮助提高内容的质量。",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=4,
    summary_method="last_msg"
)

Critic (to Writer):


        撰写一篇简洁而引人入胜的博客文章，主题为
       DeepLearning.AI。确保博客文章在
       100个字以内。
       

--------------------------------------------------------------------------------
Writer (to Critic):

标题：探索DeepLearning.AI：智能的未来

在DeepLearning.AI的引领下，人工智能的边界正不断扩展。这个平台通过提供前沿的课程和资源，帮助学习者掌握深度学习的精髓。无论你是AI新手还是专家，DeepLearning.AI都为你提供了通往智能未来的桥梁。加入我们，一起揭开AI的神秘面纱，探索技术的无限可能。

--------------------------------------------------------------------------------
Critic (to Writer):

标题：DeepLearning.AI：智能未来之钥

DeepLearning.AI，人工智能的先锋，通过其前沿课程和资源，正引领我们深入探索深度学习的奥秘。无论初学者或专家，这里都是通往智能未来的桥梁。加入我们，一同揭开AI的神秘面纱，探索技术的无限可能。

--------------------------------------------------------------------------------
Writer (to Critic):

标题：DeepLearning.AI：解锁智能未来

DeepLearning.AI是人工智能领域的灯塔，以其尖端课程和资源，引领我们深入深度学习的精髓。无论你是AI新手还是资深专家，这里都是你通往智能未来的关键。加入我们，一起揭开AI的神秘面纱，探索科技的无限潜能。

--------------------------------------------------------------------------------
Critic (to Writer):

标题：DeepLearning.AI：智能未来的钥匙

DeepLearning.AI，人工智能的领航

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="你是一位SEO审查员，以优化内容以适应搜索引擎而闻名，"
        "确保内容排名良好并吸引自然流量。"
        "请确保你的建议简洁（不超过3个要点），"
        "具体且切中要点。"
        "开始审查时请说明你的角色。",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="你是一位法律审查员，以确保内容符合法律规定而闻名，"
        "确保内容不违反任何潜在的法律问题。"
        "请确保你的建议简洁（不超过3个要点），"
        "具体且切中要点。"
        "开始审查时请说明你的角色。",
)


In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="你是一位伦理审查员，以确保内容符合伦理规定而闻名，"
        "确保内容不违反任何潜在的伦理问题。"
        "请确保你的建议简洁（不超过3个要点），"
        "具体且切中要点。"
        "开始审查时请说明你的角色。",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="你是一位元审查员，你聚合和审查其他审查员的作品，并给出最终的建议。",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''审查以下内容。 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "仅以JSON对象返回审查结果:"
        "{'审查员': '', '审查': ''}. 这里的审查员应为你的角色",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "仅以JSON对象返回审查结果:"
        "{'审查员': '', '审查': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "仅以JSON对象返回审查结果:"
        "{'审查员': '', '审查': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "汇总所有审查员的反馈并给出最终的写作建议。", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        撰写一篇简洁而引人入胜的博客文章，主题为
       DeepLearning.AI。确保博客文章在
       100个字以内。
       

--------------------------------------------------------------------------------
Writer (to Critic):

标题：探索DeepLearning.AI：智能的未来

在DeepLearning.AI的引领下，人工智能的边界正不断扩展。这个平台通过提供前沿的课程和资源，帮助学习者掌握深度学习的精髓。无论你是AI新手还是专家，DeepLearning.AI都为你提供了通往智能未来的桥梁。加入我们，一起揭开AI的神秘面纱，探索技术的无限可能。

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to SEO Reviewer):

审查以下内容。 
            

 标题：探索DeepLearning.AI：智能的未来

在DeepLearning.AI的引领下，人工智能的边界正不断扩展。这个平台通过提供前沿的课程和资源，帮助学习者掌握深度学习的精髓。无论你是AI新手还是专家，DeepLearning.AI都为你提供了通往智能未来的桥梁。加入我们，一起揭开AI的神秘面纱，探索技术的无限可能。

--------------------------------------------------------------------------------
SEO Reviewer (to Critic):

作为SEO审查员，我将提供以下优化建议以提升内容在搜索引擎

## Get the summary

In [15]:
print(res.summary)

```json
{
  "最终建议": "综合所有审查员的反馈，建议进行以下优化：
  1. **关键词优化**：确保内容中的关键词布局合理，以提升搜索引擎的表现。
  2. **元描述优化**：优化元描述，使其更具吸引力，从而吸引更多自然流量。
  3. **内部链接策略**：建立有效的内部链接策略，提高网站内部页面的互连性。
  4. **法律合规性**：确保品牌使用合法，避免误导性声明，并严格遵守隐私和数据保护法规。
  5. **伦理合规性**：确保宣传内容真实，不夸大其词，尊重用户隐私，并避免使用可能引起歧视或偏见的语言。"
}
```
